In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mplt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from IPython.core.interactiveshell import InteractiveShell
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import explained_variance_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
import matplotlib.dates as mdates
%matplotlib inline
import seaborn as sns
import math
import gc
import ipaddress
from urllib.parse import urlparse
from tldextract import extract
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

from data_science_utils import dataframe as df_utils
from data_science_utils import models as model_utils
from data_science_utils.dataframe import column as column_utils


from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,classification_report

import lightgbm as lgb

np.set_printoptions(threshold=np.nan)



plt.rcParams["figure.figsize"] = (24,4)

from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score
import missingno as msno
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA
import datetime
from scipy import signal
import matplotlib.pyplot as plt
from datetime import timedelta
from sklearn import linear_model



InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_seq_items', None)
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import warnings
warnings.filterwarnings('ignore')
import dask.dataframe as dd

In [2]:
df_train = pd.read_hdf("data/train_1.h5","table")

df_pos = df_train[df_train['is_attributed']==1]
df_negs = df_train[df_train['is_attributed']==0]
df_negs = df_negs.sample(frac=0.6)
gc.collect()


27

In [5]:
df_test = pd.read_hdf("data/test_1.h5","table")

In [3]:

        

df_train = pd.concat([df_pos,df_negs],ignore_index=True)
df_train.shape
df_pos = None
df_negs = None
del df_pos
del df_negs


gc.collect()

(111125072, 9)

0

In [4]:
def split_dataframe(df,splits=[0.5,0.4,0.1]):
    if(abs(np.sum(splits)-1)>1e-9):
        raise ValueError("sum of splits should be 1")
    probs = np.random.rand(len(df))
    lower = 0
    upper = 0
    dfs = list()
    for i in splits:
        upper = lower+i
        mask = (probs>=lower) & (probs < upper)
        print("lower: %s, upper: %s mask len: %s"%(lower,upper,np.sum(mask)))
        df_new = df[mask]
        dfs.append(df_new)
        lower = upper
    return dfs
gc.collect()

60

In [5]:
splits = split_dataframe(df_train)
df_train = splits[0]
df_cv1 = splits[1]
df_cv2 = splits[2]
gc.collect()

lower: 0, upper: 0.5 mask len: 55571007
lower: 0.5, upper: 0.9 mask len: 44438695
lower: 0.9, upper: 1.0 mask len: 11115370


0

In [6]:
gc.collect()

0

In [7]:
# Define all the groupby transformations
GROUPBY_AGGREGATIONS = [
    
    # Frequency of is_attributed in different groups
    {'groupby': ['ip'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['app'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['device'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['os'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['channel'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['hour'], 'select': 'is_attributed', 'agg': 'mean'},
    
    
    
    {'groupby': ['ip','app'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['ip','hour'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['ip','device'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['ip','channel'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['app','device'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['app','hour'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['app','os'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['app','channel'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['os','channel'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['os','hour'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['device','channel'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['channel','hour'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['device','hour'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['ip','app','device'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['ip','app','channel'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['app','device','os'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['app','channel','os'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['ip','os','channel'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['ip','os','hour'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['ip','app','hour'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['app','os','hour'], 'select': 'is_attributed', 'agg': 'mean'}
    
]


In [8]:
# Define all the groupby transformations
GROUPBY_AGGREGATIONS_v2 = [
    
    # Frequency of is_attributed in different groups
    {'groupby': ['ip','device','hour'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['ip','channel','hour'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['device','channel','hour'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['app','channel','hour'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['os','channel','hour'], 'select': 'is_attributed', 'agg': 'mean'},
    
    {'groupby': ['ip','app','device','hour'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['ip','app','device','channel'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['ip','app','os','device','channel'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['ip','app','os','device','channel','hour'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['ip','app','device','channel','hour'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['app','device','channel'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['app','os','device','channel'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['app','device','channel','hour'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['app','os','device','channel','hour'], 'select': 'is_attributed', 'agg': 'mean'},
    {'groupby': ['app','os','channel','hour'], 'select': 'is_attributed', 'agg': 'mean'},
]


In [9]:
def add_groupby_specs(GROUPBY_AGGREGATIONS):
    gc.collect()
    
    for spec in GROUPBY_AGGREGATIONS:
        power = 5-len(spec['groupby'])
        log_group = np.log(pow(10,power))
        min_examples_for_validity = 20
        # Name of the aggregation we're applying
        agg_name = spec['agg']

        # Info
        print("Grouping by {}, and aggregating {} with {}".format(
            spec['groupby'], spec['select'], agg_name
        ))

        # Unique list of features to select
        all_features = list(set(spec['groupby'] + [spec['select']]))

        # Name of new feature
        new_feature = '{}_{}_{}'.format('_'.join(spec['groupby']), agg_name, spec['select'])

        # Perform the groupby
        group = df_train[all_features].groupby(spec['groupby'])[spec['select']]
        
        gp = group.transform(spec['agg'])
        gp_count = group.transform('count')
        c1 = gp_count<=min_examples_for_validity
        gp_count[c1] = np.nan
        
        
        gp[c1] = np.nan
        
        
        conf = np.log1p(gp_count) / log_group
        conf[conf<1] = 1
        gp = gp*conf
        df_train[new_feature] = gp
        gp = None
        del gp
        
        
#         gpt = group.agg({'mean':np.mean,'count':lambda x: x.shape[0]}).reset_index()
#         gpt['conf'] = np.log1p(gpt['count']) / log_group
#         gpt[new_feature] = gpt['mean']*gpt['conf']
        
        new_feature_count = new_feature+'_count'
        gpt = group.agg(spec['agg']).reset_index().rename(index=str, columns={spec['select']: new_feature})
        gpt_count = group.agg('count').reset_index().rename(index=str, columns={spec['select']: new_feature_count})[new_feature_count]
        
        c1 = gpt_count<=min_examples_for_validity
        

        gpt_count[c1] = np.nan
        
        gptfn=gpt[new_feature]
        gptfn[c1] = np.nan
        gpt[new_feature] = gptfn

        conf = np.log1p(gpt_count) / log_group
        conf[conf<1] = 1
        gpt[new_feature] = gpt[new_feature]*conf
        conf = None
        
        gp_count = None
        group = None
        gpt_count = None
        del group

        # Merge back to X_train
        # df_train[new_feature] = df_train[spec['groupby']].merge(gpt, on=spec['groupby'], how='left')[new_feature]
        df_test[new_feature] = pd.merge(df_test[spec['groupby']],gpt,how='left',on=spec['groupby'])[new_feature].values
        gc.collect()
        df_cv1[new_feature] = pd.merge(df_cv1[spec['groupby']],gpt,how='left',on=spec['groupby'])[new_feature].values
        gc.collect()
        df_cv2[new_feature] = pd.merge(df_cv2[spec['groupby']],gpt,how='left',on=spec['groupby'])[new_feature].values
        gc.collect()
        
        

In [10]:
gc.collect()

0

In [11]:

# df_train=df_train[df_train['ip']==53672]
# df_cv1=df_cv1[df_cv1['ip']==53672]
# df_test=df_test[df_test['ip']==53672]
# add_groupby_specs([{'groupby': ['ip'], 'select': 'is_attributed', 'agg': 'mean'}])


# df_train[df_train['ip']==53672].shape

# df_train[(df_train['ip']==53672)&(df_train['is_attributed']==1)].shape

# df_cv1[df_cv1['ip']==53672].shape

# df_cv1[(df_cv1['ip']==53672)&(df_cv1['is_attributed']==1)].shape

# df_train[df_train['ip']==53672].sample(10)


# df_cv1[df_cv1['ip']==53672].sample(10)


In [12]:
gc.collect()
add_groupby_specs(GROUPBY_AGGREGATIONS)
gc.collect()
add_groupby_specs(GROUPBY_AGGREGATIONS_v2)
gc.collect()

0

Grouping by ['ip'], and aggregating is_attributed with mean
Grouping by ['app'], and aggregating is_attributed with mean
Grouping by ['device'], and aggregating is_attributed with mean
Grouping by ['os'], and aggregating is_attributed with mean
Grouping by ['channel'], and aggregating is_attributed with mean
Grouping by ['hour'], and aggregating is_attributed with mean
Grouping by ['ip', 'app'], and aggregating is_attributed with mean
Grouping by ['ip', 'hour'], and aggregating is_attributed with mean
Grouping by ['ip', 'device'], and aggregating is_attributed with mean
Grouping by ['ip', 'channel'], and aggregating is_attributed with mean
Grouping by ['app', 'device'], and aggregating is_attributed with mean
Grouping by ['app', 'hour'], and aggregating is_attributed with mean
Grouping by ['app', 'os'], and aggregating is_attributed with mean
Grouping by ['app', 'channel'], and aggregating is_attributed with mean
Grouping by ['os', 'channel'], and aggregating is_attributed with mean
Gr

31

Grouping by ['ip', 'device', 'hour'], and aggregating is_attributed with mean
Grouping by ['ip', 'channel', 'hour'], and aggregating is_attributed with mean
Grouping by ['device', 'channel', 'hour'], and aggregating is_attributed with mean
Grouping by ['app', 'channel', 'hour'], and aggregating is_attributed with mean
Grouping by ['os', 'channel', 'hour'], and aggregating is_attributed with mean


31

In [13]:
# Define all the groupby transformations
cols = ['ip','app','device','os','channel','hour']
GROUPBY_AGGREGATIONS_NO_SCALING = [
    
    
    # V1 - GroupBy Features #
    #########################    

    # Variance in hour, for ip-app-os
    {'groupby': ['ip','app','os'], 'select': 'hour', 'agg': 'var'},
    # Variance in hour, for ip-day-channel
    {'groupby': ['ip','channel'], 'select': 'hour', 'agg': 'var'},
    
    
    
    {'groupby': ['app','device'], 'select': 'hour', 'agg': 'var'},
    
    {'groupby': ['device','os'], 'select': 'hour', 'agg': 'var'},
    
    {'groupby': ['device'], 'select': 'hour', 'agg': 'var'},
    
    {'groupby': ['ip'], 'select': 'hour', 'agg': 'var'},
    
    

    {'groupby': ['ip','hour'], 'select': 'is_attributed', 'agg': 'count'},
    # Count, for ip-app
    {'groupby': ['ip', 'app'], 'select': 'is_attributed', 'agg': 'count'},        
    # Count, for ip-app-os
    {'groupby': ['ip', 'app', 'os'], 'select': 'is_attributed', 'agg': 'count'},
    # Count, for ip-app-hour
    {'groupby': ['ip','app','hour'], 'select': 'is_attributed', 'agg': 'count'},
    
    {'groupby': ['app','hour'], 'select': 'is_attributed', 'agg': 'count'},
    
    {'groupby': ['ip','device','hour'], 'select': 'is_attributed', 'agg': 'count'},
    
    {'groupby': ['device','hour'], 'select': 'is_attributed', 'agg': 'count'},
    
    {'groupby': ['device'], 'select': 'is_attributed', 'agg': 'count'},
    
    {'groupby': ['os','hour'], 'select': 'is_attributed', 'agg': 'count'},
    # Mean hour, for ip-app-channel
    {'groupby': ['ip','app','channel'], 'select': 'hour', 'agg': 'mean'},
    {'groupby': ['ip','app','channel'], 'select': 'hour', 'agg': 'var'},
    
    {'groupby': ['ip','app','channel'], 'select': 'is_attributed', 'agg': 'count'},
    
    {'groupby': ['app','channel'], 'select': 'hour', 'agg': 'mean'},
    
    {'groupby': ['ip'], 'select': 'hour', 'agg': 'mean'},
    
    {'groupby': ['device','app'], 'select': 'hour', 'agg': 'mean'},
    
    {'groupby': ['device','os'], 'select': 'hour', 'agg': 'mean'},
    
    # V2 - GroupBy Features #
    #########################
    # Average clicks on app by distinct users; is it an app they return to?
    {'groupby': ['app'], 
     'select': 'ip', 
     'agg': lambda x: float(len(x)) / len(x.unique()), 
     'agg_name': 'AvgViewPerDistinct'
    },
    # How popular is the app or channel?
    {'groupby': ['app'], 'select': 'is_attributed', 'agg': 'count'},
    {'groupby': ['channel'], 'select': 'is_attributed', 'agg': 'count'},
    
    {'groupby': ['ip'], 'select': 'is_attributed', 'agg': 'count'},
    {'groupby': ['device'], 'select': 'is_attributed', 'agg': 'count'},
    
    {'groupby': ['device'], 'select': 'is_attributed', 'agg': 'count'},
    
    {'groupby': ['os'], 'select': 'is_attributed', 'agg': 'count'},
    
    {'groupby': ['app'], 'select': 'is_attributed', 'agg': 'count'},
    
    {'groupby': ['hour'], 'select': 'is_attributed', 'agg': 'count'},
    
    # V3 - GroupBy Features                                              #
    # https://www.kaggle.com/bk0000/non-blending-lightgbm-model-lb-0-977 #
    ###################################################################### 
    {'groupby': ['ip'], 'select': 'channel', 'agg': 'nunique'}, 
    {'groupby': ['ip'], 'select': 'app', 'agg': 'nunique'}, 

    {'groupby': ['ip','app'], 'select': 'os', 'agg': 'nunique'}, 
    {'groupby': ['ip'], 'select': 'device', 'agg': 'nunique'}, 
    {'groupby': ['app'], 'select': 'channel', 'agg': 'nunique'}, 
    {'groupby': ['ip', 'device', 'os'], 'select': 'app', 'agg': 'nunique'},
    
    {'groupby': ['channel'], 'select': 'ip', 'agg': 'nunique'}, 
    {'groupby': ['app'], 'select': 'ip', 'agg': 'nunique'}, 
    {'groupby': ['hour'], 'select': 'ip', 'agg': 'nunique'}, 
    {'groupby': ['os','app'], 'select': 'ip', 'agg': 'nunique'}, 
    {'groupby': ['device'], 'select': 'ip', 'agg': 'nunique'}, 
    {'groupby': ['channel'], 'select': 'app', 'agg': 'nunique'}, 
    {'groupby': ['app', 'device', 'os'], 'select': 'ip', 'agg': 'nunique'},
    
    {'groupby': ['channel'], 'select': 'os', 'agg': 'nunique'}, 
    {'groupby': ['app'], 'select': 'channel', 'agg': 'nunique'},
    {'groupby': ['app'], 'select': 'device', 'agg': 'nunique'},
    {'groupby': ['hour'], 'select': 'device', 'agg': 'nunique'}, 
    {'groupby': ['os','app'], 'select': 'device', 'agg': 'nunique'}, 
    {'groupby': ['device'], 'select': 'channel', 'agg': 'nunique'}, 
    
    

    {'groupby': ['device'], 'select': 'app', 'agg': 'nunique'},
    
    {'groupby': ['device'], 'select': 'os', 'agg': 'nunique'},
    
    {'groupby': ['device'], 'select': 'hour', 'agg': 'nunique'},
    
    {'groupby': ['os'], 'select': 'app', 'agg': 'nunique'},
    
    {'groupby': ['app'], 'select': 'os', 'agg': 'nunique'},
    
    {'groupby': ['app','hour'], 'select': 'ip', 'agg': 'nunique'},
    
    {'groupby': ['device','app','hour'], 'select': 'ip', 'agg': 'nunique'},
    
    {'groupby': ['channel','app','hour'], 'select': 'ip', 'agg': 'nunique'},
    
    {'groupby': ['channel','app','device','hour'], 'select': 'ip', 'agg': 'nunique'},
    
    {'groupby': ['channel','app','device'], 'select': 'ip', 'agg': 'nunique'},
    
    {'groupby': ['channel','app'], 'select': 'ip', 'agg': 'nunique'},
    
    {'groupby': ['device','app'], 'select': 'ip', 'agg': 'nunique'},
    
    {'groupby': ['ip','app','hour'], 'select': 'device', 'agg': 'nunique'},
    
    {'groupby': ['hour'], 'select': 'os', 'agg': 'nunique'},
    {'groupby': ['hour'], 'select': 'channel', 'agg': 'nunique'},
    
    {'groupby': ['hour'], 'select': 'app', 'agg': 'nunique'}
    

]


In [14]:
def add_groupby_specs_no_scaling(GROUPBY_AGGREGATIONS):
    gc.collect()
    
    for spec in GROUPBY_AGGREGATIONS:
        min_examples_for_validity = 5
        # Name of the aggregation we're applying
        agg_name = spec['agg_name'] if 'agg_name' in spec else spec['agg']

        # Info
        print("Grouping by {}, and aggregating {} with {}".format(
            spec['groupby'], spec['select'], agg_name
        ))

        # Unique list of features to select
        all_features = list(set(spec['groupby'] + [spec['select']]))

        # Name of new feature
        new_feature = '{}_{}_{}'.format('_'.join(spec['groupby']), agg_name, spec['select'])

        # Perform the groupby
        group = df_train[all_features].groupby(spec['groupby'])[spec['select']]
        
        gp = group.transform(spec['agg'])
        #gp_count = group.transform('count')
        #c1 = gp_count<=min_examples_for_validity
        #gp_count[c1] = np.nan
        
        
        #gp[c1] = np.nan
        
        
        #conf = np.log1p(gp_count) / log_group
        # conf[conf<0.15] = 0.15
        #gp = gp*conf
        df_train[new_feature] = gp
        gp = None
        del gp
        
        
        # new_feature_count = new_feature+'_count'
        gpt = group.agg(spec['agg']).reset_index().rename(index=str, columns={spec['select']: new_feature})
        #gpt_count = group.agg('count').reset_index().rename(index=str, columns={spec['select']: new_feature_count})[new_feature_count]
        #c1 = gpt_count<=min_examples_for_validity
        #gpt_count[c1] = np.nan
        #gptfn=gpt[new_feature]
        #gptfn[c1] = np.nan
        #gpt[new_feature] = gptfn

        #conf = np.log1p(gpt_count) / log_group
        # conf[conf<0.15] = 0.15
        #gpt[new_feature] = gpt[new_feature]*conf
        #conf = None
        
        #gp_count = None
        group = None
        #gpt_count = None
        del group

        # Merge back to X_train
        # df_train[new_feature] = df_train[spec['groupby']].merge(gpt, on=spec['groupby'], how='left')[new_feature]
        df_test[new_feature] = pd.merge(df_test[spec['groupby']],gpt,how='left',on=spec['groupby'])[new_feature].values
        gc.collect()
        df_cv1[new_feature] = pd.merge(df_cv1[spec['groupby']],gpt,how='left',on=spec['groupby'])[new_feature].values
        gc.collect()
        df_cv2[new_feature] = pd.merge(df_cv2[spec['groupby']],gpt,how='left',on=spec['groupby'])[new_feature].values

        gc.collect()

In [15]:
gc.collect()
add_groupby_specs_no_scaling(GROUPBY_AGGREGATIONS_NO_SCALING)
gc.collect()

581

Grouping by ['ip', 'app', 'os'], and aggregating hour with var
Grouping by ['ip', 'channel'], and aggregating hour with var
Grouping by ['app', 'device'], and aggregating hour with var
Grouping by ['device', 'os'], and aggregating hour with var
Grouping by ['device'], and aggregating hour with var
Grouping by ['ip'], and aggregating hour with var
Grouping by ['ip', 'hour'], and aggregating is_attributed with count
Grouping by ['ip', 'app'], and aggregating is_attributed with count
Grouping by ['ip', 'app', 'os'], and aggregating is_attributed with count
Grouping by ['ip', 'app', 'hour'], and aggregating is_attributed with count
Grouping by ['app', 'hour'], and aggregating is_attributed with count
Grouping by ['ip', 'device', 'hour'], and aggregating is_attributed with count
Grouping by ['device', 'hour'], and aggregating is_attributed with count
Grouping by ['device'], and aggregating is_attributed with count
Grouping by ['os', 'hour'], and aggregating is_attributed with count
Grouping

7

In [21]:
gc.collect()
df_cv1.to_hdf('data/cv1_2.h5',"table",mode='w')
gc.collect()
print("cv1_2_2.h5 done")

7

0

cv1_2_2.h5 done


In [22]:
df_cv1 = None
del df_cv1

In [23]:
gc.collect()
df_cv2.to_hdf('data/cv2_2.h5',"table",mode='w')
gc.collect()
print("cv2_2_2.h5 done")

0

0

cv2_2_2.h5 done


In [24]:
df_cv2 = None
gc.collect()
del df_cv2


0

In [25]:
gc.collect()
df_test.to_hdf('data/test_2.h5',"table",mode='w')
gc.collect()
print("Test_2.h5 done")

0

0

Test_2.h5 done


In [26]:
df_test = None
gc.collect()
del df_test
gc.collect()

25

0

In [27]:
gc.collect()
df_train.to_hdf('data/train_2.h5',"table",mode='w')
gc.collect()
print("Train_2.h5 done")

0

0

Train_2.h5 done


In [ ]:
df_train.groupby(['is_attributed']).mean()
df_cv1.groupby(['is_attributed']).mean()
# df_cv2.groupby(['is_attributed']).mean()


In [ ]:
gc.collect()
df_train.shape

In [ ]:
df_utils.count_nulls(df_train)

In [122]:
gc.collect()

18